# Cleaning Your Data

Vamos pegar um registro de acesso à web e descobrir a partir dele as páginas mais visualizadas de um site! Parece fácil, certo?

Vamos configurar uma regex que nos permita analisar uma linha de log de acesso do Apache:

In [11]:
import re  # Importa a biblioteca de expressões regulares

# Compila uma expressão regular para capturar dados de um log de acesso de servidor web
format_pat = re.compile(
    r"(?P<host>[\d\.]+)\s"             # Captura o endereço IP do cliente (ex: 192.168.1.1)
    r"(?P<identity>\S*)\s"             # Captura a identidade do cliente (geralmente um hífen)
    r"(?P<user>\S*)\s"                 # Captura o nome de usuário se autenticado (geralmente um hífen)
    r"\[(?P<time>.*?)\]\s"              # Captura a hora do acesso, entre colchetes (ex: [01/Jan/2024:00:00:01 +0000])
    r'"(?P<request>.*?)"\s'             # Captura a solicitação HTTP (ex: "GET / HTTP/1.1")
    r"(?P<status>\d+)\s"                # Captura o código de status HTTP (ex: 200)
    r"(?P<bytes>\S*)\s"                 # Captura o número de bytes enviados (geralmente um hífen se não houver dados)
    r'"(?P<referer>.*?)"\s'             # Captura a URL referida (de onde o usuário veio, pode ser um hífen)
    r'"(?P<user_agent>.*?)"\s*'         # Captura a informação do agente do usuário (navegador, sistema operacional, etc.)
)


Aqui está o caminho para o arquivo de log que estou analisando:

In [12]:
logPath = "access_log.txt"

Agora vamos preparar um pequeno script para extrair a URL em cada acesso e usar um dicionário para contar o número de vezes que cada uma aparece. Em seguida, classificaremos e imprimiremos as 20 páginas principais. O que poderia dar errado?

In [13]:
# Dicionário para armazenar a contagem de acessos por URL
URLCounts = {}

# Abre o arquivo de log no modo de leitura
with open(logPath, "r") as f:
    # Itera sobre cada linha do arquivo, removendo espaços em branco à direita
    for line in (l.rstrip() for l in f):
        # Tenta fazer o match da linha com a expressão regular definida anteriormente
        match = format_pat.match(line)
        if match:  # Se houver uma correspondência
            access = match.groupdict()  # Obtém os dados correspondentes como um dicionário
            request = access['request']  # Extrai a solicitação HTTP
            
            # Divide a solicitação nos seus componentes (ação, URL e protocolo)
            (action, URL, protocol) = request.split()
            
            # Verifica se a URL já está no dicionário
            if URL in URLCounts:
                URLCounts[URL] = URLCounts[URL] + 1  # Incrementa a contagem da URL
            else:
                URLCounts[URL] = 1  # Inicializa a contagem da URL

# Classifica as URLs com base em suas contagens, do maior para o menor
results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

# Imprime as 20 URLs mais acessadas e suas contagens
for result in results[:20]:
    print(result + ": " + str(URLCounts[result]))


ValueError: not enough values to unpack (expected 3, got 1)

Hum. A parte 'solicitação' da linha deve ser parecida com isto:

OBTER /blog/HTTP/1.1

Deve haver uma ação HTTP, a URL e o protocolo. Mas parece que isso nem sempre acontece. Vamos imprimir solicitações que não contenham três itens:

In [14]:
# Dicionário para armazenar a contagem de acessos por URL
URLCounts = {}

# Abre o arquivo de log no modo de leitura
with open(logPath, "r") as f:
    # Itera sobre cada linha do arquivo, removendo espaços em branco à direita
    for line in (l.rstrip() for l in f):
        # Tenta fazer o match da linha com a expressão regular definida anteriormente
        match = format_pat.match(line)
        if match:  # Se houver uma correspondência
            access = match.groupdict()  # Obtém os dados correspondentes como um dicionário
            request = access['request']  # Extrai a solicitação HTTP
            
            # Divide a solicitação em seus componentes
            fields = request.split()
            
            # Verifica se o número de campos extraídos é diferente de 3
            if (len(fields) != 3):
                print(fields)  # Imprime os campos se não corresponderem ao esperado


['_\\xb0ZP\\x07tR\\xe5']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


Huh. Além dos campos vazios, existe um que contém apenas lixo. Bem, vamos modificar nosso script para verificar esse caso:

In [15]:
# Dicionário para armazenar a contagem de acessos por URL
URLCounts = {}

# Abre o arquivo de log no modo de leitura
with open(logPath, "r") as f:
    # Itera sobre cada linha do arquivo, removendo espaços em branco à direita
    for line in (l.rstrip() for l in f):
        # Tenta fazer o match da linha com a expressão regular definida anteriormente
        match = format_pat.match(line)
        if match:  # Se houver uma correspondência
            access = match.groupdict()  # Obtém os dados correspondentes como um dicionário
            request = access['request']  # Extrai a solicitação HTTP
            
            # Divide a solicitação em seus componentes
            fields = request.split()
            
            # Verifica se o número de campos extraídos é igual a 3
            if (len(fields) == 3):
                URL = fields[1]  # A URL é o segundo campo da solicitação
                
                # Atualiza a contagem de acessos para a URL
                if URL in URLCounts:
                    URLCounts[URL] = URLCounts[URL] + 1  # Incrementa a contagem existente
                else:
                    URLCounts[URL] = 1  # Inicializa a contagem se a URL ainda não estiver no dicionário

# Ordena as URLs por contagem de acessos em ordem decrescente
results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

# Imprime as 20 URLs mais acessadas e suas contagens
for result in results[:20]:
    print(result + ": " + str(URLCounts[result]))


/xmlrpc.php: 68494
/wp-login.php: 1923
/: 440
/blog/: 138
/robots.txt: 123
/sitemap_index.xml: 118
/post-sitemap.xml: 118
/page-sitemap.xml: 117
/category-sitemap.xml: 117
/orlando-headlines/: 95
/san-jose-headlines/: 85
http://51.254.206.142/httptest.php: 81
/comics-2/: 76
/travel/: 74
/entertainment/: 72
/business/: 70
/national/: 70
/national-headlines/: 70
/world/: 70
/weather/: 70


Funcionou! Mas os resultados realmente não fazem sentido. O que realmente queremos são páginas acessadas por humanos reais em busca de notícias do nosso pequeno site de notícias. O que diabos é xmlrpc.php? Uma olhada no próprio log revela muitas entradas como esta:

46.166.139.20 - - [05/Dez/2015:05:19:35 +0000] "POST /xmlrpc.php HTTP/1.0" 200 370 "-" "Mozilla/4.0 (compatível: MSIE 7.0; Windows NT 6.0)"

Não tenho muita certeza do que o script faz, mas indica que não estamos apenas processando ações GET. Não queremos POSTS, então vamos filtrá-los:

In [17]:
# Inicializa um dicionário para contar o número de acessos a cada URL
URLCounts = {}

# Abre o arquivo de log para leitura
with open(logPath, "r") as f:
    # Itera sobre cada linha do arquivo, removendo espaços em branco à direita
    for line in (l.rstrip() for l in f):
        # Tenta fazer uma correspondência da linha com o padrão definido
        match = format_pat.match(line)
        if match:
            # Se a linha corresponder ao padrão, armazena os grupos correspondentes em um dicionário
            access = match.groupdict()
            request = access['request']  # Extrai a parte da solicitação da linha do log
            fields = request.split()  # Divide a solicitação em seus componentes (ação, URL, protocolo)

            # Verifica se a solicitação tem exatamente 3 campos
            if (len(fields) == 3):
                (action, URL, protocol) = fields  # Desempacota os campos em variáveis separadas

                # Conta apenas as solicitações do tipo GET
                if (action == 'GET'):
                    # Se a URL já estiver no dicionário, incrementa sua contagem
                    if URL in URLCounts:
                        URLCounts[URL] = URLCounts[URL] + 1
                    else:
                        # Caso contrário, inicializa a contagem da URL em 1
                        URLCounts[URL] = 1

# Ordena as URLs com base na contagem de acessos, do maior para o menor
results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

# Imprime as 20 URLs mais acessadas junto com suas contagens
for result in results[:20]:
    print(result + ": " + str(URLCounts[result]))


/: 434
/blog/: 138
/robots.txt: 123
/sitemap_index.xml: 118
/post-sitemap.xml: 118
/page-sitemap.xml: 117
/category-sitemap.xml: 117
/orlando-headlines/: 95
/san-jose-headlines/: 85
http://51.254.206.142/httptest.php: 81
/comics-2/: 76
/travel/: 74
/entertainment/: 72
/business/: 70
/national/: 70
/national-headlines/: 70
/world/: 70
/weather/: 70
/about/: 69
/defense-sticking-head-sand/: 69


Isso está começando a parecer melhor. Mas este é um site de notícias - as pessoas estão realmente lendo o pequeno blog nele em vez de páginas de notícias? Isso não faz sentido. Vejamos uma entrada /blog/ típica no log:

54.165.199.171 - - [05/dez/2015:09:32:05 +0000] "OBTER /blog/ HTTP/1.0" 200 31670 "-" "-"

Hum. Por que o agente do usuário está em branco? Parece algum tipo de raspador malicioso ou algo assim. Vamos descobrir com quais agentes de usuário estamos lidando:

In [18]:
# Inicializa um dicionário para contar o número de acessos de cada agente de usuário
UserAgents = {}

# Abre o arquivo de log para leitura
with open(logPath, "r") as f:
    # Itera sobre cada linha do arquivo, removendo espaços em branco à direita
    for line in (l.rstrip() for l in f):
        # Tenta fazer uma correspondência da linha com o padrão definido
        match = format_pat.match(line)
        if match:
            # Se a linha corresponder ao padrão, armazena os grupos correspondentes em um dicionário
            access = match.groupdict()
            agent = access['user_agent']  # Extrai o agente de usuário da linha do log

            # Atualiza a contagem do agente de usuário no dicionário
            if agent in UserAgents:
                UserAgents[agent] = UserAgents[agent] + 1  # Incrementa a contagem se já existir
            else:
                UserAgents[agent] = 1  # Inicializa a contagem em 1 se for a primeira vez

# Ordena os agentes de usuário com base na contagem de acessos, do maior para o menor
results = sorted(UserAgents, key=lambda i: int(UserAgents[i]), reverse=True)

# Imprime todos os agentes de usuário junto com suas contagens
for result in results:
    print(result + ": " + str(UserAgents[result]))


Mozilla/4.0 (compatible: MSIE 7.0; Windows NT 6.0): 68484
-: 4035
Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.0): 1724
W3 Total Cache/0.9.4.1: 468
Mozilla/5.0 (compatible; Baiduspider/2.0; +http://www.baidu.com/search/spider.html): 278
Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html): 248
Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36: 158
Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0: 144
Mozilla/5.0 (iPad; CPU OS 8_4 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12H143 Safari/600.1.4: 120
Mozilla/5.0 (Linux; Android 5.1.1; SM-G900T Build/LMY47X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.76 Mobile Safari/537.36: 47
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm): 43
Mozilla/5.0 (compatible; MJ12bot/v1.4.5; http://www.majestic12.co.uk/bot.php?+): 41
Opera/9.80 (Windows NT 6.0) Presto/2.12.388 Version/12.1

Caramba! Além de '-', há também um milhão de robôs web diferentes acessando o site e poluindo meus dados. Filtrar todos eles é realmente difícil, mas livrar-se daqueles que poluem significativamente meus dados, neste caso, deveria ser uma questão de se livrar de '-', qualquer coisa que contenha "bot" ou "spider" e W3 Total Cache.

In [19]:
# Inicializa um dicionário para contar o número de acessos de cada URL
URLCounts = {}

# Abre o arquivo de log para leitura
with open(logPath, "r") as f:
    # Itera sobre cada linha do arquivo, removendo espaços em branco à direita
    for line in (l.rstrip() for l in f):
        # Tenta fazer uma correspondência da linha com o padrão definido
        match = format_pat.match(line)
        if match:
            # Se a linha corresponder ao padrão, armazena os grupos correspondentes em um dicionário
            access = match.groupdict()
            agent = access['user_agent']  # Extrai o agente de usuário da linha do log

            # Verifica se o agente de usuário não é um bot ou spider
            if (not('bot' in agent or 'spider' in agent or 
                    'Bot' in agent or 'Spider' in agent or
                    'W3 Total Cache' in agent or agent == '-')):
                
                # Extrai a requisição da linha do log
                request = access['request']
                fields = request.split()
                
                # Verifica se a requisição contém 3 campos (método, URL e protocolo)
                if (len(fields) == 3):
                    (action, URL, protocol) = fields
                    
                    # Conta apenas requisições do tipo 'GET'
                    if (action == 'GET'):
                        # Incrementa a contagem da URL no dicionário
                        if URL in URLCounts:
                            URLCounts[URL] = URLCounts[URL] + 1  # Incrementa se já existir
                        else:
                            URLCounts[URL] = 1  # Inicializa em 1 se for a primeira vez

# Ordena as URLs com base na contagem de acessos, do maior para o menor
results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

# Imprime as 20 URLs mais acessadas junto com suas contagens
for result in results[:20]:
    print(result + ": " + str(URLCounts[result]))


/: 77
/orlando-headlines/: 36
/?page_id=34248: 28
/wp-content/cache/minify/000000/M9AvyUjVzUstLy7PLErVz8lMKkosqtTPKtYvTi7KLCgpBgA.js: 27
/wp-content/cache/minify/000000/M9bPKixNLarUy00szs8D0Zl5AA.js: 27
/wp-content/cache/minify/000000/lY7dDoIwDIVfiG0KxkfxfnbdKO4HuxICTy-it8Zw15PzfSftzPCckJem-x4qUWArqBPl5mygZLEgyhdOaoxToGyGaiALiOfUnIz0qDLOdSZGE-nOlpc3kopDzrSyavVVt_veb5qSDVhjsQ6dHh_B_eE_z2pYIGJ7iBWKeEio_eT9UQe4xHhDll27mGRryVu_pRc.js: 27
/wp-content/cache/minify/000000/fY45DoAwDAQ_FMvkRQgFA5ZyWLajiN9zNHR0O83MRkyt-pIctqYFJPedKyYzfHg2PzOFiENAzaD07AxcpKmTolORvDjZt8KEfhBUGjZYCf8Fb0fvA1TXCw.css: 25
/?author=1: 21
/wp-content/cache/minify/000000/hcrRCYAwDAXAhXyEjiQ1YKAh4SVSx3cE7_uG7ASr4M9qg3kGWyk1adklK84LHtRj_My6Y0Pfqcz-AA.js: 20
/wp-content/uploads/2014/11/nhn1.png: 19
/wp-includes/js/wp-emoji-release.min.js?ver=4.3.1: 17
/wp-content/cache/minify/000000/BcGBCQAgCATAiUSaKYSERPk3avzuht4SkBJnt4tHJdqgnPBqKldesTcN1R8.js: 17
/wp-login.php: 16
/comics-2/: 12
/world/: 12
/favicon.ico: 10
/wp-content/up

Agora, nosso novo problema é que estamos recebendo um monte de acessos a coisas que não são páginas da web. Não estamos interessados ​​neles, então vamos filtrar qualquer URL que não termine em / (todas as páginas do meu site são acessadas dessa maneira - novamente, isso é aplicar o conhecimento sobre meus dados à análise!)

In [20]:
# Inicializa um dicionário para contar o número de acessos de cada URL
URLCounts = {}

# Abre o arquivo de log para leitura
with open(logPath, "r") as f:
    # Itera sobre cada linha do arquivo, removendo espaços em branco à direita
    for line in (l.rstrip() for l in f):
        # Tenta fazer uma correspondência da linha com o padrão definido
        match = format_pat.match(line)
        if match:
            # Se a linha corresponder ao padrão, armazena os grupos correspondentes em um dicionário
            access = match.groupdict()
            agent = access['user_agent']  # Extrai o agente de usuário da linha do log

            # Verifica se o agente de usuário não é um bot ou spider
            if (not('bot' in agent or 'spider' in agent or 
                    'Bot' in agent or 'Spider' in agent or
                    'W3 Total Cache' in agent or agent == '-')):
                
                # Extrai a requisição da linha do log
                request = access['request']
                fields = request.split()
                
                # Verifica se a requisição contém 3 campos (método, URL e protocolo)
                if (len(fields) == 3):
                    (action, URL, protocol) = fields
                    
                    # Checa se a URL termina com uma barra ("/")
                    if (URL.endswith("/")):
                        # Conta apenas requisições do tipo 'GET'
                        if (action == 'GET'):
                            # Incrementa a contagem da URL no dicionário
                            if URL in URLCounts:
                                URLCounts[URL] = URLCounts[URL] + 1  # Incrementa se já existir
                            else:
                                URLCounts[URL] = 1  # Inicializa em 1 se for a primeira vez

# Ordena as URLs com base na contagem de acessos, do maior para o menor
results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

# Imprime as 20 URLs mais acessadas junto com suas contagens
for result in results[:20]:
    print(result + ": " + str(URLCounts[result]))


/: 77
/orlando-headlines/: 36
/comics-2/: 12
/world/: 12
/weather/: 4
/australia/: 4
/about/: 4
/national-headlines/: 3
/feed/: 2
/sample-page/feed/: 2
/science/: 2
/technology/: 2
/entertainment/: 1
/san-jose-headlines/: 1
/business/: 1
/travel/feed/: 1


Isso está começando a parecer mais verossímil! Mas se você fosse ainda mais fundo, descobriria que as páginas /feed/ são suspeitas e que alguns robôs ainda estão escapando. No entanto, é correto dizer que notícias de Orlando, notícias mundiais e quadrinhos são as páginas mais populares acessadas por um ser humano real neste dia.

A moral da história é: conheça seus dados! E sempre questione e examine seus resultados antes de tomar decisões com base neles. Se sua empresa tomar uma decisão errada porque você forneceu uma análise de dados de origem incorretos, você poderá ter sérios problemas.

Certifique-se de que as decisões tomadas durante a limpeza dos seus dados também sejam justificáveis ​​- não elimine os dados só porque eles não suportam os resultados desejados!

## Activity
Estes resultados ainda não são perfeitos; URLs que incluem “feed” não são, na verdade, páginas visualizadas por humanos. Modifique ainda mais este código para remover URLs que incluem "/feed". Melhor ainda, extraia algumas entradas de log dessas páginas e entenda de onde vêm essas visualizações.

In [23]:
from collections import defaultdict
import re

# Compilação do padrão de expressão regular para análise do log
format_pat = re.compile(
    r"(?P<host>[\d\.]+)\s"
    r"(?P<identity>\S*)\s"
    r"(?P<user>\S*)\s"
    r"\[(?P<time>.*?)\]\s"
    r'"(?P<request>.*?)"\s'
    r"(?P<status>\d+)\s"
    r"(?P<bytes>\S*)\s"
    r'"(?P<referer>.*?)"\s'
    r'"(?P<user_agent>.*?)"\s*'
)

def count_url_access(log_path):
    # Inicializa um defaultdict para contar o número de acessos de cada URL
    URLCounts = defaultdict(int)

    # Definindo palavras-chave para bots
    bot_keywords = {'bot', 'spider', 'W3 Total Cache', '-'}

    # Abre o arquivo de log para leitura
    with open(log_path, "r") as f:
        for line in (l.rstrip() for l in f):
            match = format_pat.match(line)
            if match:
                access = match.groupdict()
                agent = access['user_agent']
                
                # Verifica se o agente de usuário é um bot ou spider
                if not any(keyword in agent for keyword in bot_keywords):
                    request = access['request']
                    fields = request.split()
                    
                    # Verifica se a requisição contém 3 campos
                    if len(fields) == 3:
                        action, URL, protocol = fields
                        
                        # Checa se a URL termina com uma barra ("/") e se é uma requisição GET
                        if URL.endswith("/") and action == 'GET':
                            URLCounts[URL] += 1  # Incrementa a contagem

    # Ordena as URLs com base na contagem de acessos
    results = sorted(URLCounts.items(), key=lambda item: item[1], reverse=True)

    # Imprime as 20 URLs mais acessadas junto com suas contagens
    for result in results[:20]:
        print(f"{result[0]}: {result[1]}")

# Chamada da função com o caminho do log
count_url_access(logPath)


/: 64
/orlando-headlines/: 29
/world/: 11
/comics-2/: 10
/weather/: 4
/australia/: 4
/about/: 4
/national-headlines/: 3
/feed/: 2
/sample-page/feed/: 2
/science/: 2
/technology/: 2
/entertainment/: 1
/san-jose-headlines/: 1
/business/: 1
/travel/feed/: 1


In [21]:
# Inicializa um dicionário para contar o número de acessos de cada URL
URLCounts = {}
feed_entries = []  # Lista para armazenar entradas de log relacionadas a URLs que contêm "/feed"

# Abre o arquivo de log para leitura
with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f):
        match = format_pat.match(line)
        if match:
            access = match.groupdict()
            agent = access['user_agent']

            # Verifica se o agente de usuário não é um bot ou spider
            if (not ('bot' in agent or 'spider' in agent or 
                      'Bot' in agent or 'Spider' in agent or
                      'W3 Total Cache' in agent or agent == '-')):
                
                request = access['request']
                fields = request.split()
                
                if len(fields) == 3:
                    action, URL, protocol = fields
                    
                    # Checa se a URL termina com uma barra ("/")
                    if URL.endswith("/"):
                        # Exclui URLs que contêm "/feed"
                        if "/feed" in URL:
                            feed_entries.append(line)  # Armazena a linha do log
                            continue  # Ignora esta URL

                        # Conta apenas requisições do tipo 'GET'
                        if action == 'GET':
                            URLCounts[URL] = URLCounts.get(URL, 0) + 1  # Incrementa ou inicializa

# Ordena as URLs com base na contagem de acessos, do maior para o menor
results = sorted(URLCounts.items(), key=lambda item: item[1], reverse=True)

# Imprime as 20 URLs mais acessadas junto com suas contagens
for result in results[:20]:
    print(f"{result[0]}: {result[1]}")

# Se houver entradas de log relacionadas a "/feed", imprime algumas delas
if feed_entries:
    print("\nEntradas de log relacionadas a URLs que contêm '/feed':")
    for entry in feed_entries[:5]:  # Exibe as primeiras 5 entradas
        print(entry)
else:
    print("\nNenhuma entrada de log relacionada a URLs que contêm '/feed'.")


/: 77
/orlando-headlines/: 36
/comics-2/: 12
/world/: 12
/weather/: 4
/australia/: 4
/about/: 4
/national-headlines/: 3
/science/: 2
/technology/: 2
/entertainment/: 1
/san-jose-headlines/: 1
/business/: 1

Entradas de log relacionadas a URLs que contêm '/feed':
195.154.250.88 - - [29/Nov/2015:22:35:35 +0000] "GET /feed/ HTTP/1.1" 200 17592 "http://www.nohatenews.com/?feed=rss2" "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.0)"
62.210.215.118 - - [01/Dec/2015:03:43:43 +0000] "GET /sample-page/feed/ HTTP/1.1" 500 200 "-" "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.66 Safari/537.36"
62.210.215.118 - - [04/Dec/2015:16:34:59 +0000] "GET /feed/ HTTP/1.1" 200 4922 "-" "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.66 Safari/537.36"
62.210.215.118 - - [04/Dec/2015:20:41:26 +0000] "GET /sample-page/feed/ HTTP/1.1" 500 200 "-" "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.15

In [27]:
import re

# Inicializa um dicionário para contar o número de acessos de cada URL
URLCounts = {}

# Abre o arquivo de log para leitura
with open(logPath, "r") as f:
    # Itera sobre cada linha do arquivo, removendo espaços em branco à direita
    for line in (l.rstrip() for l in f):
        # Tenta fazer uma correspondência da linha com o padrão definido
        match = format_pat.match(line)
        if match:
            # Se a linha corresponder ao padrão, armazena os grupos correspondentes em um dicionário
            access = match.groupdict()
            agent = access['user_agent']  # Extrai o agente de usuário da linha do log
            
            # Verifica se o agente de usuário não é um bot ou spider, e não está vazio
            if (not('bot' in agent.lower() or 'spider' in agent.lower() or 
                    'w3 total cache' in agent or agent == '-')):
                
                # Extrai a requisição da linha do log
                request = access['request']
                fields = request.split()
                
                # Verifica se a requisição contém 3 campos (método, URL e protocolo)
                if (len(fields) == 3):
                    (action, URL, protocol) = fields
                    
                    # Checa se a URL termina com uma barra ("/") e não contém "/feed"
                    if (URL.endswith("/") and "/feed" not in URL):
                        # Conta apenas requisições do tipo 'GET'
                        if (action == 'GET'):
                            # Incrementa a contagem da URL no dicionário
                            if URL in URLCounts:
                                URLCounts[URL] += 1  # Incrementa se já existir
                            else:
                                URLCounts[URL] = 1  # Inicializa em 1 se for a primeira vez

# Ordena as URLs com base na contagem de acessos, do maior para o menor
results = sorted(URLCounts.items(), key=lambda item: item[1], reverse=True)

# Imprime as 20 URLs mais acessadas junto com suas contagens
for result in results[:20]:
    print(f"{result[0]}: {result[1]}")


/: 77
/orlando-headlines/: 36
/comics-2/: 12
/world/: 12
/weather/: 4
/australia/: 4
/about/: 4
/national-headlines/: 3
/science/: 2
/technology/: 2
/entertainment/: 1
/san-jose-headlines/: 1
/business/: 1
